In [1]:
import re
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer

## Regex for extracting the numbers

In [2]:
Text={"orders":[{"id":1},{"id":2},{"id":3},{"id":4},{"id":5},{"id":6},{"id":7},{"id":8},{"id":9},{"id":10},{"id":11},{"id":648},{"id":649},{"id":650},{"id":651},{"id":652},{"id":653}],"errors":[{"code":3,"message":"[PHP Warning #2] count(): Parameter must be an array or an object that implements Countable (153)"}]}
Text=str(Text)

In [3]:
re.findall(':\s?(\d+)',Text)

['1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '648',
 '649',
 '650',
 '651',
 '652',
 '653',
 '3']

## Google Apps  Dataset Analysis

In [4]:
df = pd.read_csv('C:/Users/faurd/Downloads/chrome_reviews.csv')

In [5]:
df.head()

,ID,Review URL,Text,Star,Thumbs Up,User Name,Developer Reply,Version,Review Date,App ID
0,3886,https://play.google.com/store/apps/details?id=...,This is very helpfull aap.,5,0,INDIAN Knowledge,NaN,83.0.4103.106,2020-12-19,com.android.chrome
1,3887,https://play.google.com/store/apps/details?id=...,Good,3,2,Ijeoma Happiness,NaN,85.0.4183.127,2020-12-19,com.android.chrome
2,3888,https://play.google.com/store/apps/details?id=...,Not able to update. Neither able to uninstall.,1,0,Priti D BtCFs-29,NaN,85.0.4183.127,2020-12-19,com.android.chrome
3,3889,https://play.google.com/store/apps/details?id=...,Nice app,4,0,Ajeet Raja,NaN,77.0.3865.116,2020-12-19,com.android.chrome
4,3890,https://play.google.com/store/apps/details?id=...,Many unwanted ads,1,0,Rams Mp,NaN,87.0.4280.66,2020-12-19,com.android.chrome


In [6]:
data=pd.DataFrame(df,columns=['Text','Star'])

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7204 entries, 0 to 7203
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    7203 non-null   object
 1   Star    7204 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 112.7+ KB


In [8]:
data['Text'] =[re.sub(r'[^\w\s]','',str(i)).lower() for i in data['Text']]

In [9]:
l=['0','1','2','3','4','5']
ind=data[~data['Star'].isin(l)].index
data.drop(ind,axis=0,inplace=True)

In [10]:
data['Star']

0       5
1       3
2       1
3       4
4       1
       ..
7199    5
7200    1
7201    5
7202    5
7203    1
Name: Star, Length: 7204, dtype: int64

In [11]:
data['Star'].value_counts()

5    3871
1    1894
4     652
3     451
2     336
Name: Star, dtype: int64

In [12]:
data['Star'].astype('int32')

0       5
1       3
2       1
3       4
4       1
       ..
7199    5
7200    1
7201    5
7202    5
7203    1
Name: Star, Length: 7204, dtype: int32

In [13]:
stop_words = set(stopwords.words('english'))
def remove_StopWord(word):
    word_tokens =  word_tokenize(str(word))
    filtered_sentence = [] 
    for w in word_tokens: 
        if w.lower() not in stop_words: 
             filtered_sentence.append(w) 
    removed = " ".join(filtered_sentence)
    return removed

In [14]:
data['Text']=data['Text'].apply(remove_StopWord)

In [15]:
data['Text']

0                                            helpfull aap
1                                                    good
2                      able update neither able uninstall
3                                                nice app
4                                       many unwanted ads
                              ...                        
7199                                              bagusss
7200                                          bad version
7201    one thing say cant spelled good dont problem t...
7202                                            excellent
7203       update lag always slow goes googleee want work
Name: Text, Length: 7204, dtype: object

In [16]:
data['Text']=data['Text'].apply(lambda x:word_tokenize(x))
data['sentiment']=data['Star'].apply(lambda x: 1 if int(x)>=3 else 0)

In [17]:
porter_stemmer = PorterStemmer()
def stemming(text):
    stem_text = [porter_stemmer.stem(word) for word in text]
    removed = " ".join(stem_text)
    return removed

In [18]:
data['Text']=data['Text'].apply(stemming)

In [19]:
data['Text'].head()

0                       helpful aap
1                              good
2    abl updat neither abl uninstal
3                          nice app
4                    mani unwant ad
Name: Text, dtype: object

In [20]:
from sklearn.feature_extraction.text import CountVectorizer
 
vectorizer = CountVectorizer(min_df =100)
# Fit the bag-of-words model
#
bag = vectorizer.fit_transform(data["Text"])

#
# Get unique words / tokens found in all the documents. The unique words / tokens represents
# the features
#
print(vectorizer.get_feature_names())
#
# Associate the indices with each unique word
#
print(vectorizer.vocabulary_)
#
# Print the numerical feature vector
#
print(bag.toarray())

['abl', 'app', 'bad', 'best', 'browser', 'cant', 'chrome', 'dont', 'download', 'even', 'excel', 'fix', 'get', 'go', 'good', 'googl', 'great', 'help', 'im', 'keep', 'like', 'love', 'need', 'new', 'nice', 'ok', 'one', 'open', 'phone', 'pleas', 'problem', 'super', 'tab', 'thank', 'time', 'tri', 'unabl', 'updat', 'use', 'wont', 'work']
{'good': 14, 'abl': 0, 'updat': 37, 'nice': 24, 'app': 1, 'bad': 2, 'time': 34, 'tri': 35, 'use': 38, 'get': 12, 'open': 27, 'chrome': 6, 'like': 20, 'thank': 33, 'download': 8, 'best': 3, 'im': 18, 'need': 22, 'even': 9, 'keep': 19, 'browser': 4, 'ok': 25, 'fix': 11, 'phone': 28, 'cant': 5, 'unabl': 36, 'googl': 15, 'problem': 30, 'go': 13, 'tab': 32, 'wont': 39, 'work': 40, 'dont': 7, 'pleas': 29, 'new': 23, 'one': 26, 'excel': 10, 'love': 21, 'super': 31, 'help': 17, 'great': 16}
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [2 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]]


In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(data['Text']) 

tfidf = TfidfVectorizer(lowercase=False,)
tfidf.fit_transform(questions)

# dict key:word and value:tf-idf score
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

In [22]:
import spacy
from tqdm import tqdm
import en_core_web_sm

In [23]:
# en_vectors_web_lg, which includes over 1 million unique vectors.
nlp = en_core_web_sm.load()

vecs1 = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu1 in tqdm(list(data['Text'])):
    doc1 = nlp(qu1) 
    # 384 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), 300])
    for word1 in doc1:
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1[7] * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
data['Text_feats_m'] = list(vecs1)

  1%|▌                                                                               | 55/7204 [00:00<01:38, 72.24it/s]<ipython-input-23-210bef18897f>:21: RuntimeWarning: Mean of empty slice.
  mean_vec1 = mean_vec1.mean(axis=0)
C:\Users\faurd\anaconda3\lib\site-packages\numpy\core\_methods.py:162: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
100%|█████████████████████████████████████████████████████████████████████████████| 7204/7204 [01:11<00:00, 101.09it/s]


In [24]:
data['Text_feats_m']

0       [0.9334196598501276, 0.9334196598501276, 0.933...
1       [-0.1161411074706854, -0.1161411074706854, -0....
2       [-11.77696897371657, -11.77696897371657, -11.7...
3       [-2.593191720828227, -2.593191720828227, -2.59...
4       [8.688050144758654, 8.688050144758654, 8.68805...
                              ...                        
7199    [7.077503779421425, 7.077503779421425, 7.07750...
7200    [1.8401771587511973, 1.8401771587511973, 1.840...
7201    [20.417079681965923, 20.417079681965923, 20.41...
7202    [1.93207671287025, 1.93207671287025, 1.9320767...
7203    [2.7957891382074003, 2.7957891382074003, 2.795...
Name: Text_feats_m, Length: 7204, dtype: object

In [25]:
data1 = data["Star"]

In [26]:
data2 = pd.DataFrame(data.Text_feats_m.values.tolist(), index= data.index)


In [27]:
data2

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.933420,0.933420,0.933420,0.933420,0.933420,0.933420,0.933420,0.933420,0.933420,0.933420,...,0.933420,0.933420,0.933420,0.933420,0.933420,0.933420,0.933420,0.933420,0.933420,0.933420
1,-0.116141,-0.116141,-0.116141,-0.116141,-0.116141,-0.116141,-0.116141,-0.116141,-0.116141,-0.116141,...,-0.116141,-0.116141,-0.116141,-0.116141,-0.116141,-0.116141,-0.116141,-0.116141,-0.116141,-0.116141
2,-11.776969,-11.776969,-11.776969,-11.776969,-11.776969,-11.776969,-11.776969,-11.776969,-11.776969,-11.776969,...,-11.776969,-11.776969,-11.776969,-11.776969,-11.776969,-11.776969,-11.776969,-11.776969,-11.776969,-11.776969
3,-2.593192,-2.593192,-2.593192,-2.593192,-2.593192,-2.593192,-2.593192,-2.593192,-2.593192,-2.593192,...,-2.593192,-2.593192,-2.593192,-2.593192,-2.593192,-2.593192,-2.593192,-2.593192,-2.593192,-2.593192
4,8.688050,8.688050,8.688050,8.688050,8.688050,8.688050,8.688050,8.688050,8.688050,8.688050,...,8.688050,8.688050,8.688050,8.688050,8.688050,8.688050,8.688050,8.688050,8.688050,8.688050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7199,7.077504,7.077504,7.077504,7.077504,7.077504,7.077504,7.077504,7.077504,7.077504,7.077504,...,7.077504,7.077504,7.077504,7.077504,7.077504,7.077504,7.077504,7.077504,7.077504,7.077504
7200,1.840177,1.840177,1.840177,1.840177,1.840177,1.840177,1.840177,1.840177,1.840177,1.840177,...,1.840177,1.840177,1.840177,1.840177,1.840177,1.840177,1.840177,1.840177,1.840177,1.840177
7201,20.417080,20.417080,20.417080,20.417080,20.417080,20.417080,20.417080,20.417080,20.417080,20.417080,...,20.417080,20.417080,20.417080,20.417080,20.417080,20.417080,20.417080,20.417080,20.417080,20.417080
7202,1.932077,1.932077,1.932077,1.932077,1.932077,1.932077,1.932077,1.932077,1.932077,1.932077,...,1.932077,1.932077,1.932077,1.932077,1.932077,1.932077,1.932077,1.932077,1.932077,1.932077


In [28]:
data3 = data2.join(data1,how='left')


In [29]:
data3

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,Star
0,0.933420,0.933420,0.933420,0.933420,0.933420,0.933420,0.933420,0.933420,0.933420,0.933420,...,0.933420,0.933420,0.933420,0.933420,0.933420,0.933420,0.933420,0.933420,0.933420,5
1,-0.116141,-0.116141,-0.116141,-0.116141,-0.116141,-0.116141,-0.116141,-0.116141,-0.116141,-0.116141,...,-0.116141,-0.116141,-0.116141,-0.116141,-0.116141,-0.116141,-0.116141,-0.116141,-0.116141,3
2,-11.776969,-11.776969,-11.776969,-11.776969,-11.776969,-11.776969,-11.776969,-11.776969,-11.776969,-11.776969,...,-11.776969,-11.776969,-11.776969,-11.776969,-11.776969,-11.776969,-11.776969,-11.776969,-11.776969,1
3,-2.593192,-2.593192,-2.593192,-2.593192,-2.593192,-2.593192,-2.593192,-2.593192,-2.593192,-2.593192,...,-2.593192,-2.593192,-2.593192,-2.593192,-2.593192,-2.593192,-2.593192,-2.593192,-2.593192,4
4,8.688050,8.688050,8.688050,8.688050,8.688050,8.688050,8.688050,8.688050,8.688050,8.688050,...,8.688050,8.688050,8.688050,8.688050,8.688050,8.688050,8.688050,8.688050,8.688050,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7199,7.077504,7.077504,7.077504,7.077504,7.077504,7.077504,7.077504,7.077504,7.077504,7.077504,...,7.077504,7.077504,7.077504,7.077504,7.077504,7.077504,7.077504,7.077504,7.077504,5
7200,1.840177,1.840177,1.840177,1.840177,1.840177,1.840177,1.840177,1.840177,1.840177,1.840177,...,1.840177,1.840177,1.840177,1.840177,1.840177,1.840177,1.840177,1.840177,1.840177,1
7201,20.417080,20.417080,20.417080,20.417080,20.417080,20.417080,20.417080,20.417080,20.417080,20.417080,...,20.417080,20.417080,20.417080,20.417080,20.417080,20.417080,20.417080,20.417080,20.417080,5
7202,1.932077,1.932077,1.932077,1.932077,1.932077,1.932077,1.932077,1.932077,1.932077,1.932077,...,1.932077,1.932077,1.932077,1.932077,1.932077,1.932077,1.932077,1.932077,1.932077,5


In [30]:
X = data.drop('Star',axis = 1)
Y = data['Star']

In [31]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.3,random_state=20) 

# Model Building

In [32]:
from sklearn.linear_model import LogisticRegression
lr_model=LogisticRegression()
lr_model.fit(x_train,y_train)
lr_model_p=lr_model.predict(x_test)

In [33]:
confusion_matrix(y_test,lr_model_p)

array([[1530,  699],
       [ 569, 4403]], dtype=int64)

In [34]:
print(classification_report(y_test,lr_model_p))

              precision    recall  f1-score   support

           0       0.73      0.69      0.71      2229
           1       0.86      0.89      0.87      4972

    accuracy                           0.82      7201
   macro avg       0.80      0.79      0.79      7201
weighted avg       0.82      0.82      0.82      7201



In [35]:
from sklearn.tree import DecisionTreeClassifier

In [36]:
dt_model=DecisionTreeClassifier(criterion='entropy')
dt_model.fit(x_train,y_train)

DecisionTreeClassifier(criterion='entropy')

In [37]:
dt_model_p=dt_model.predict(x_test)
confusion_matrix(y_test,dt_model_p)

array([[1147,  514],
       [1082, 4458]], dtype=int64)

In [38]:
print(classification_report(y_test,dt_model_p))

              precision    recall  f1-score   support

           0       0.69      0.51      0.59      2229
           1       0.80      0.90      0.85      4972

    accuracy                           0.78      7201
   macro avg       0.75      0.71      0.72      7201
weighted avg       0.77      0.78      0.77      7201



In [39]:
from sklearn.ensemble import RandomForestClassifier

In [40]:
rf_model=RandomForestClassifier(n_estimators=100)
rf_model.fit(x_train,y_train) 

RandomForestClassifier()

In [41]:
rf_model_p=rf_model.predict(x_test)

In [42]:
confusion_matrix(y_test,rf_model_p)

array([[1546,  537],
       [ 683, 4435]], dtype=int64)

In [43]:
print(classification_report(y_test,rf_model_p))

              precision    recall  f1-score   support

           0       0.74      0.69      0.72      2229
           1       0.87      0.89      0.88      4972

    accuracy                           0.83      7201
   macro avg       0.80      0.79      0.80      7201
weighted avg       0.83      0.83      0.83      7201



In [77]:
chrome_df['sentiment_pred']=rf_model_p

## Ranking Analysis

In [44]:
ranking_df=pd.read_csv('Downloads/browser_rankings_data.csv')

In [45]:
ranking_df.dropna(axis=0,inplace=True)
ranking_df.head()

,ID,Keyword,Rank,Country,Language,Date,App ID,Date of Last Description Change,Short Description,Long Description
0,1,browser,2.0,in,en,16-12-2020,net.fast.web.browser,18-12-2020,"The small, fastest, secure Web Browser, best m...","One of the smallest, fastest, secure Web Brows..."
1,2,browser,3.0,in,en,16-12-2020,com.android.chrome,18-12-2020,"Fast, simple, and secure. Google Chrome browse...","Google Chrome is a fast, easy to use, and secu..."
2,3,browser,4.0,in,en,16-12-2020,com.opera.browser,27-02-2021,"Fast, safe web browser: free VPN, Ad blocker, ...","Fast, safe and private, introducing the latest..."
3,3,browser,4.0,in,en,16-12-2020,com.opera.browser,18-12-2020,"Fast, safe web browser: free VPN, Ad blocker, ...","Opera is a fast, safe web browser for your And..."
4,4,browser,1.0,in,en,16-12-2020,com.cloudmosa.puffinTV,07-02-2021,Puffin TV Browser - One App to Watch All Video...,Puffin TV Browser is a special edition of Puff...


In [46]:
ranking_df.dtypes

ID                                   int64
Keyword                             object
Rank                               float64
Country                             object
Language                            object
Date                                object
App ID                              object
Date of Last Description Change     object
Short Description                   object
Long Description                    object
dtype: object

In [47]:
import random
sample1=random.sample(range(0,ranking_df.shape[0]),300)

In [48]:
sample_df=pd.DataFrame(ranking_df,index=sample1)

In [49]:
sample_df['Short Description'] =[re.sub(r'[^\w\s]','',str(i)).lower() for i in sample_df['Short Description']]
sample_df['Long Description'] =[re.sub(r'[^\w\s]','',str(i)).lower() for i in sample_df['Long Description']]

In [50]:
table1=pd.crosstab(sample_df['Short Description'],sample_df['Long Description'])

In [51]:
from scipy.stats import chi2_contingency
stat,p_value1,dof,expected1=chi2_contingency(table1)

In [52]:
alpha = 0.05
if p_value1<=alpha:
    print("Reject H0,There is a relationship between 2 categorical variables")
else:
    print("Retain H0,There is no relationship between 2 categorical variables")

Reject H0,There is a relationship between 2 categorical variables


In [53]:
table2=pd.crosstab(sample_df['Short Description'],sample_df['Rank'])

In [54]:
stat,p_value2,dof,expected2=chi2_contingency(table2)

In [55]:
if p_value2<=alpha:
    print("Reject H0,There is a relationship between 2 categorical variables")
else:
    print("Retain H0,There is no relationship between 2 categorical variables")

Reject H0,There is a relationship between 2 categorical variables


In [56]:
table3=pd.crosstab(sample_df['Long Description'],sample_df['Rank'])

In [57]:
stat,p_value3,dof,expected3=chi2_contingency(table3)

In [58]:
if p_value3<=alpha:
    print("Reject H0,There is a relationship between 2 categorical variables")
else:
    print("Retain H0,There is no relationship between 2 categorical variables")

Reject H0,There is a relationship between 2 categorical variables


In [59]:
rank=pd.crosstab(ranking_df['Rank'],ranking_df['App ID'])

In [60]:
rank[:50]

App ID,com.android.chrome,com.brave.browser,com.cloudmosa.puffinTV,com.duckduckgo.mobile.android,com.opera.browser,com.transsion.phoenix,com.vivaldi.browser,net.fast.web.browser
Rank,,,,,,,,
1.0,18,5,72,428,8,0,0,31
2.0,6,201,212,12,38,0,0,21
3.0,7,57,80,0,62,0,0,8
4.0,6,39,76,0,44,0,0,5
5.0,1,20,32,12,50,0,0,10
6.0,5,23,36,104,34,6,0,5
7.0,4,9,40,24,26,11,0,8
8.0,11,10,8,10,16,15,0,12
9.0,1,2,16,2,12,15,0,8


## Grammar Correction

In [61]:
grammar_df=pd.read_csv('Downloads/review_data.csv')
grammar_df

,text,star,app_id,reviewDate
0,Anathi Khanyile,5,com.fingersoft.hillclimb,18/03/21
1,Tony bahut funny hai Hill climbing racing my f...,5,com.fingersoft.hillclimb,18/03/21
2,Teturwu,1,com.fingersoft.hillclimb,18/03/21
3,Hoooooooooooyaaaaaaaaa what a game hoooooooooo...,5,com.fingersoft.hillclimb,18/03/21
4,This game is nice,5,com.fingersoft.hillclimb,18/03/21
...,...,...,...,...
29995,Plz my Vidos viral,5,video.like,21/03/21
29996,Nice,5,video.like,21/03/21
29997,Disturbing too much as YouTube ad,1,video.like,21/03/21
29998,Koub valo vai,5,video.like,21/03/21


In [62]:
grammar_df.shape

(30000, 4)

In [63]:
grammar_df['text'].value_counts()

Good                                             1582
Nice                                              708
Excellent                                         263
Very good                                         226
good                                              218
                                                 ... 
Forced to give access to storage permission .       1
It's the best game ever I love playing it           1
Some grab drivers are rude                          1
Great. Have used it long time                       1
Phone start lagging while using this app            1
Name: text, Length: 21559, dtype: int64

In [64]:
grammar_df['text'] =[re.sub(r'[^\w\s]','',str(i)).lower() for i in grammar_df['text']]

In [65]:
grammar_df.drop(grammar_df[grammar_df['text']==''].index,axis=0,inplace=True)

In [66]:
from happytransformer import HappyTextToText

In [67]:
happy_text=HappyTextToText('T5','vennify/t5-base-grammar-correction')

04/11/2022 22:41:42 - INFO - happytransformer.happy_transformer -   Using model: cpu


In [68]:
from happytransformer import TTSettings

In [69]:
arg=TTSettings(num_beams=5,min_length=1)
i=0
def pred(tex):
    global i
    tet='grammar:'+tex
    result=happy_text.generate_text(tet,args=arg)
    print(i)
    i+=1
    if tex.lower()+'.'==result.text.lower():
        return 0
    return 1

In [70]:
col=grammar_df['text'][:100]
grammar_error=col.apply(pred)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [71]:
grammar_error

0      0
1      1
2      0
3      1
4      0
      ..
99     0
100    1
103    0
104    0
105    1
Name: text, Length: 100, dtype: int64